# Rf 6 0 4_Constraints

Likelihood and minimization: fitting with constraints




**Author:** Clemens Lange, Wouter Verkerke (C++ version)  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:58 AM.</small></i>

In [1]:
from __future__ import print_function
import ROOT

Welcome to JupyROOT 6.18/03


Create model and dataset
----------------------------------------------

Construct a Gaussian p.d.f

In [2]:
x = ROOT.RooRealVar("x", "x", -10, 10)

m = ROOT.RooRealVar("m", "m", 0, -10, 10)
s = ROOT.RooRealVar("s", "s", 2, 0.1, 10)
gauss = ROOT.RooGaussian("gauss", "gauss(x,m,s)", x, m, s)


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Construct a flat p.d.f (polynomial of 0th order)

In [3]:
poly = ROOT.RooPolynomial("poly", "poly(x)", x)

model = f*gauss + (1-f)*poly

In [4]:
f = ROOT.RooRealVar("f", "f", 0.5, 0., 1.)
model = ROOT.RooAddPdf(
    "model",
    "model",
    ROOT.RooArgList(
        gauss,
        poly),
    ROOT.RooArgList(f))

Generate small dataset for use in fitting below

In [5]:
d = model.generate(ROOT.RooArgSet(x), 50)

Create constraint pdf
-----------------------------------------

Construct Gaussian constraint p.d.f on parameter f at 0.8 with
resolution of 0.1

In [6]:
fconstraint = ROOT.RooGaussian(
    "fconstraint",
    "fconstraint",
    f,
    ROOT.RooFit.RooConst(0.8),
    ROOT.RooFit.RooConst(0.1))

Method 1 - add internal constraint to model
-------------------------------------------------------------------------------------

Multiply constraint term with regular p.d.f using ROOT.RooProdPdf
Specify in fitTo() that internal constraints on parameter f should be
used

Multiply constraint with p.d.f

In [7]:
modelc = ROOT.RooProdPdf(
    "modelc", "model with constraint", ROOT.RooArgList(model, fconstraint))

Fit model (without use of constraint term)

In [8]:
r1 = model.fitTo(d, ROOT.RooFit.Save())

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (poly)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 f            5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 m            0.00000e+00  2.00000e+00   -1.00000e+01  1.00000e+01
     3 s            2.00000e+00  9.50000e-01    1.00000e-01  1.00000e+01
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1500           

Fit modelc with constraint term on parameter f

In [9]:
r2 = modelc.fitTo(
    d,
    ROOT.RooFit.Constrain(
        ROOT.RooArgSet(f)),
    ROOT.RooFit.Save())

[#1] INFO:Minization --  Including the following contraint terms in minimization: (fconstraint)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_modelc_modelData_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (poly)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss)
 **********
 **   10 **SET PRINT           1
 **********
 **********
 **   11 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 f            6.49867e-01  1.21452e-01    0.00000e+00  1.00000e+00
     2 m            7.18243e-01  4.96642e-01   -1.00000e+01  1.00000e+01
     3 s            2.18801e+00  4.61289e-01    1.00000e-01  1.00000e+01
 **********
 **   12 **SET ERR         0.5
 ****

Method 2 - specify external constraint when fitting
------------------------------------------------------------------------------------------

Construct another Gaussian constraint p.d.f on parameter f at 0.8 with
resolution of 0.1

In [10]:
fconstext = ROOT.RooGaussian("fconstext", "fconstext", f, ROOT.RooFit.RooConst(
    0.2), ROOT.RooFit.RooConst(0.1))

Fit with external constraint

In [11]:
r3 = model.fitTo(d, ROOT.RooFit.ExternalConstraints(
    ROOT.RooArgSet(fconstext)), ROOT.RooFit.Save())

[#1] INFO:Minization --  Including the following contraint terms in minimization: (fconstext)
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_modelData_with_constr) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (poly)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gauss)
 **********
 **   19 **SET PRINT           1
 **********
 **********
 **   20 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 f            7.37845e-01  7.53286e-02    0.00000e+00  1.00000e+00
     2 m            6.63502e-01  5.04046e-01   -1.00000e+01  1.00000e+01
     3 s            2.38847e+00  4.94571e-01    1.00000e-01  1.00000e+01
 **********
 **   21 **SET ERR         0.5
 *******

Print the fit results

In [12]:
print("fit result without constraint (data generated at f=0.5)")
r1.Print("v")
print("fit result with internal constraint (data generated at f=0.5, is f=0.8+/-0.2)")
r2.Print("v")
print("fit result with (another) external constraint (data generated at f=0.5, is f=0.2+/-0.1)")
r3.Print("v")

fit result without constraint (data generated at f=0.5)
fit result with internal constraint (data generated at f=0.5, is f=0.8+/-0.2)
fit result with (another) external constraint (data generated at f=0.5, is f=0.2+/-0.1)

  RooFitResult: minimized FCN value: 134.849, estimated distance to minimum: 4.23315e-05
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter  InitialValue    FinalValue +/-  Error     GblCorr.
  --------------------  ------------  --------------------------  --------
                     f    5.0000e-01    6.4987e-01 +/-  1.21e-01  <none>
                     m    0.0000e+00    7.1824e-01 +/-  4.97e-01  <none>
                     s    2.0000e+00    2.1880e+00 +/-  4.61e-01  <none>


  RooFitResult: minimized FCN value: 133.901, estimated distance to minimum: 1.19334e-05
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIM